In [1]:
from utils.imports import *
from utils.preproc_utils import *

Using TensorFlow backend.


In [2]:
def get_image(output,img_array, patient_id, node_idx, new_x, new_y, new_z, cube_radius, i, rand):
    trainX_1 = img_array[new_z - cube_radius: new_z + cube_radius,
               new_y - cube_radius: new_y + cube_radius,
               new_x - cube_radius: new_x + cube_radius]

    if trainX_1.shape == (cube_radius * 2, cube_radius * 2, cube_radius * 2):
        trainX_1 = resize(trainX_1, [36, 36, 36])
        np.save(
            output + str(cube_radius * 2) + ('/') + str(patient_id) + '_' + str(node_idx) + '_' + str(i) + '.npy',
            trainX_1)
        # cv2.imwrite(output_true+str(cube_radius*2)+('/') + str(patient_id)+'_'+str(node_idx1)+str('_2')+ '.png',trainX_1[17]*255) 
        np.save(output + str(cube_radius * 2) + ('/') + str(patient_id) + '_' + str(node_idx) + '_' + str(
            i + 1) + '.npy', trainX_1[:, :, ::-1])
        # cv2.imwrite(output_true+str(cube_radius*2)+('/') + str(patient_id)+'_'+str(node_idx1)+str('_3')+ '.png',trainX_1[:, :, ::-1][17]*255)              
    if rand > 0:
        for j in range(rand):
            new_z1 = new_z + random.choice([ -1, 1])
            new_y1 = new_y + random.choice([-2, -1, 1, 2])
            new_x1 = new_x + random.choice([-2, -1, 1, 2])
            trainX_2 = img_array[new_z1 - cube_radius: new_z1 + cube_radius,
                       new_y1 - cube_radius: new_y1 + cube_radius,
                       new_x1 - cube_radius: new_x1 + cube_radius]
            if trainX_2.shape == (cube_radius * 2, cube_radius * 2, cube_radius * 2):
                trainX_2 = resize(trainX_2, [36, 36, 36])
                np.save(output + str(cube_radius * 2) + ('/') + str(patient_id) + '_' + str(node_idx) + '_' + str(
                    i+2) + '_' + str(j) + '.npy', trainX_2)
                # cv2.imwrite(output_true+str(cube_radius*2)+('/') + str(patient_id)+'_'+str(node_idx1)+str('_0')+ '.png',trainX_2[17]*255) 
                np.save(output + str(cube_radius * 2) + ('/') + str(patient_id) + '_' + str(node_idx) + '_' + str(
                    i+3) + '_' + str(j) + '.npy', trainX_2[:, :, ::-1])
                # cv2.imwrite(output_true+str(cube_radius*2)+('/') + str(patient_id)+'_'+str(node_idx1)+str('_1')+ '.png',trainX_2[:, :, ::-1][17]*255)                                 


def create_cls_sample(df_anno,df_pred,img_file,data_path,output_true,output_false):
    mini_df_anno = df_anno[df_anno["file"]==img_file] #get all nodules associate with file
    mini_df_pred = df_pred[df_pred["file"]==img_file]
    if mini_df_anno.shape[0]>0: # some files may not have a nodule--skipping those 
        patient_id = img_file[:-9]
        img_array = np.load(data_path + img_file)
        #img_array = normalize(img_array)
        
        img_array = img_array.clip(-1000,400)
        img_array = np.expand_dims(img_array,1)
        img_array = np.squeeze(my_PreProc(img_array))
        img_array = img_array/255.
        img_array[img_array > 1.] = 1.
        img_array[img_array < 0.] = 0.
        pos_annos = pd.read_csv(data_path + img_file[:-9] + '_annos_pos.csv')
        origin = np.array([pos_annos.loc[0]['origin_x'],pos_annos.loc[0]['origin_y'],pos_annos.loc[0]['origin_z']]) 
        spacing = np.array([pos_annos.loc[0]['spacing_x'],pos_annos.loc[0]['spacing_y'],pos_annos.loc[0]['spacing_z']]) 
        
        for node_idx1, cur_row1 in mini_df_anno.iterrows():       
            node_x = cur_row1["coordX"]
            node_y = cur_row1["coordY"]
            node_z = cur_row1["coordZ"]
            diam = cur_row1["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)            
            new_x = int(v_center[2])
            new_y = int(v_center[1])
            new_z = int(v_center[0])
       

            get_image(output_true,img_array,patient_id,node_idx1,new_x,new_y,new_z,6,0,0) 
            get_image(output_true,img_array,patient_id,node_idx1,new_x,new_y,new_z,9,4,0) 
            get_image(output_true,img_array,patient_id,node_idx1,new_x,new_y,new_z,12,8,0) 
                
        for node_idx2, cur_row2 in mini_df_pred.iterrows():       
            node_x = cur_row2["coordX"]
            node_y = cur_row2["coordY"]
            node_z = cur_row2["coordZ"]
            diam = cur_row2["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)            
            new_x0 = int(v_center[2])
            new_y0 = int(v_center[1])
            new_z0 = int(v_center[0])         

            get_image(output_false,img_array,patient_id,node_idx2,new_x0,new_y0,new_z0,6,0,0) 
            get_image(output_false,img_array,patient_id,node_idx2,new_x0,new_y0,new_z0,9,4,0) 
            get_image(output_false,img_array,patient_id,node_idx2,new_x0,new_y0,new_z0,12,8,0) 
            
    return

In [3]:
csv_path = PATH['annotations_train']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
pred_csv_path = PATH['model_train_pred']
data_path = PATH['model_train_pred']
anno_csv_new = pd.read_csv(csv_path + "annotations_all.csv")
pred_csv_new = pd.read_csv(pred_csv_path + "2pred_csv_new.csv")

In [4]:
pred_csv_new = pred_csv_new[pred_csv_new.index%4 == 0]

In [5]:
patients = [x for x in os.listdir(data_path) if 'orig.npy' in x][0:30]

In [6]:
anno_csv_new["file"] = anno_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
anno_csv_new = anno_csv_new.dropna()
pred_csv_new["file"] = pred_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
pred_csv_new = pred_csv_new.dropna()

In [7]:
Parallel(n_jobs=-1)(delayed(create_cls_sample)(anno_csv_new,pred_csv_new,patient,data_path,output_true,output_false) for patient in tqdm(sorted(patients)))

100%|██████████| 30/30 [01:32<00:00,  1.76s/it]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
delete = True
delete = False

if delete:
    for i in os.listdir(output_true):
        if i != '.DS_Store':
            for j in os.listdir(output_true+i):
                if j != '.DS_Store':
                    os.remove(output_true+i+'/'+j)
    for i in os.listdir(output_false):
        if i != '.DS_Store':
            for j in os.listdir(output_false+i):
                if j != '.DS_Store':
                    os.remove(output_false+i+'/'+j)
                    